<a href="https://colab.research.google.com/github/quinn-dougherty/DS-Unit-2-Sprint-2-Linear-Regression/blob/master/module3-gradient-descent/ASSGNMNT_Gradient_Descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradient Descent Implementation Challenge!!

## Use gradient descent to find the optimal parameters of a **multiple** regression model. (We only showed an implementation for a bivariate model during lecture.)

A note: Implementing gradient descent in any context is not trivial, particularly the step where we calculate the gradient will change based on the number of parameters that we're trying to optimize for. You will need to research what the gradient of a multiple regression model looks like. This challenge is pretty open-ended but I hope it will be thrilling. Please work together, help each other, share resources and generally expand your understanding of gradient descent as you try and achieve this implementation. 

## Suggestions:

Start off with a model that has just two $X$ variables You can use any datasets that have at least two x variables. Potential candidates might be the blood pressure dataset that we used during lecture on Monday: [HERE](https://college.cengage.com/mathematics/brase/understandable_statistics/7e/students/datasets/mlr/excel/mlr02.xls) or any of the housing datasets. You would just need to select from them the two varaibles $x$ variables and one y variable that you want to work with that you most want to work with. 

Use Sklearn to find the optimal parameters of your model first. (like we did during the lecture.) So that you can compare the parameter estimates of your gradient-descent linear regression to the estimates of OLS linear regression. If implemented correctly they should be nearly identical.

Becoming a Data Scientist is all about striking out into the unknown, getting stuck and then researching and fighting and learning until you get yourself unstuck. Work together! And fight to take your own learning-rate fueled step towards your own optimal understanding of gradient descent! 


In [215]:
##
### https://web.stanford.edu/~hastie/ElemStatLearn/

'''Prostate data info

Predictors (columns 1--8)

lcavol
lweight
age
lbph
svi
lcp
gleason
pgg45

outcome (column 9)

lpsa

train/test indicator (column 10)

This last column indicates which 67 observations were used as the 
"training set" and which 30 as the test set, as described on page 48
in the book.

There was an error in these data in the first edition of this
book. Subject 32 had a value of 6.1 for lweight, which translates to a
449 gm prostate! The correct value is 44.9 gm. We are grateful to
Prof. Stephen W. Link for alerting us to this error.

The features must first be scaled to have mean zero and  variance 96 (=n)
before the analyses in Tables 3.1 and beyond.  That is, if x is the  96 by 8 matrix
of features, we compute xp <- scale(x,TRUE,TRUE)

'''
import pandas as pd
import altair as alt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from functools import reduce


def repeatedly(f, n): 
  def composition(f,g):
    # "f after g" 
    return lambda x: f(g(x))
  return reduce(lambda f, ff: composition(f, ff), [f]*n)

cols = ['praf', 'pmek', 'plcg', 'PIP2', 'PIP3', 'p44/42', 'pakts473', 'PKA', 'PKC', 'P38', 'pjnk']
df = pd.read_csv("https://web.stanford.edu/~hastie/ElemStatLearn/datasets/prostate.data", 
                 delim_whitespace=True)

df['ones'] = np.ones(df.shape[0])

df.head()

,lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45,lpsa,train,ones
1,-0.579818,2.769459,50,-1.386294,0,-1.386294,6,0,-0.430783,T,1.0
2,-0.994252,3.319626,58,-1.386294,0,-1.386294,6,0,-0.162519,T,1.0
3,-0.510826,2.691243,74,-1.386294,0,-1.386294,7,20,-0.162519,T,1.0
4,-1.203973,3.282789,58,-1.386294,0,-1.386294,6,0,-0.162519,T,1.0
5,0.751416,3.432373,62,-1.386294,0,-1.386294,6,0,0.371564,T,1.0


In [0]:
def LR(X, y, testsize=0.3): 
  #print(X.shape, y.shape)
  # X is a dataframe with arbitrary features
  # y is a dataframe with one feature
  # they are each sliced from the same master df. 
  
  # Split into test and train datasets
  X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=testsize, random_state=42)

  # fit model using train datasets
  model = LinearRegression()
  model.fit(X_train, Y_train)

  # Create new predictions using x_test
  y_pred = model.predict(X_test)

  # Measure Accuracy using y_test and y_pred
  RMSE = (np.sqrt(mean_squared_error(Y_test, y_pred)))
  R2 = r2_score(Y_test, y_pred)

  print('RMSE is {}'.format(RMSE))
  print('R^2 is {}'.format(R2))

  print("coefficients: ", model.coef_)
  print("intercepts: ", model.intercept_)
  
  return {'RMSE': RMSE, 'R2': R2, 'coefficients': model.coef_, 'intercept': model.intercept_}

In [212]:
X = df[['ones', 'lcavol']]
y = df.lpsa

from_sklearn = LR(X.drop('ones', axis=1),y)


RMSE is 0.7856818244526866
R^2 is 0.4699726156051881
coefficients:  [0.76784083]
intercepts:  1.4138982670092684


In [0]:
epochs = 2000
theta = np.ones(X.shape[1])[:, np.newaxis]
gamma = 0.11 # learning rate. 

def yhat(theta): 
  # long form: X.ones1 * theta[0] + X.lcavol * theta[1]
  return pd.DataFrame(np.matmul(X, theta))

def cost(theta): 
  sqrd_residuals = (y[:, np.newaxis] - yhat(theta)) ** 2
  return (1/X.shape[1]) * sqrd_residuals.sum().values

def del_cost_0(theta): 
  N = X.shape[1]
  return np.divide(-2,N) * (y[:, np.newaxis] - yhat(theta)).sum()

def del_cost_1(theta): 
  N = X.shape[1]
  return np.divide(-2,N) * (X.lcavol[:, np.newaxis] * (y[:, np.newaxis] - yhat(theta))).sum()

# algorithm: repeat theta_k <- theta_k - alpha * del H / del theta_k `epochs` times, for k in range(len(theta))

def update(theta): 
  return [theta[0] - gamma * del_cost_0(theta).values, theta[1] - gamma * del_cost_1(theta).values]


sqrd_residuals = (y[:, np.newaxis] - yhat(theta)) ** 2

#np.gradient(lambda theta_0, theta_1: Hhat([theta_0, theta_1]))

X.shape, y.shape, yhat(theta).shape, sqrd_residuals.shape



In [134]:
(y[:, np.newaxis] - yhat(theta)).sum()

0    12.452598
dtype: float64

In [141]:
cost(theta)[0]

35.509789318731464

In [233]:
theta_from_sklearn = [from_sklearn['intercept'], from_sklearn['coefficients'][0]]

theta_ours = repeatedly(update, epochs//12)(theta)

#np.testing.assert_almost_equal(theta_ours, theta_from_sklearn)

list(map(lambda l: l[0], theta_ours)), theta_from_sklearn

cost(theta_ours), cost(theta_from_sklearn)

(array([inf]), array([29.65211356]))

## Stretch Goals

If you happen upon the most useful resources for accomplishing this challenge first, I want you to spend time today studying other variations of Gradient Descent-Based Optimizers.

- Try and write a function that can perform gradient descent for arbitarily large (in dimensionality) multiple regression models. 
- Create a notebook for yourself exploring these topics
- How do they differ from the "vanilla" gradient descent we explored today
- How do these different gradient descent-based optimizers seek to overcome the challenge of finding the global minimum among various local minima?
- Write a blog post that reteaches what you have learned about these other gradient descent-based optimizers.

[Overview of GD-based optimizers](http://ruder.io/optimizing-gradient-descent/)

[Siraj Raval - Evolution of Gradient Descent-Based Optimizers](https://youtu.be/nhqo0u1a6fw)

# What you need to know about derivatives for this

